Market basket analysis is used by companies to identify items that are frequently purchased together.

**How Does Market Basket Analysis Work?**

Market basket analysis is frequently used by restaurants, retail stores, and online shopping platforms to encourage customers to make more purchases in a single visit. This is a use-case of data science in marketing that increases company sales and drives business growth and commonly utilizes the Apriori algorithm.

**What is the Apriori Algorithm?**

The Apriori algorithm is the most common technique for performing market basket analysis.

It is used for association rule mining, which is a rule-based process used to identify correlations between items purchased by users.

**What Are the Components of the Apriori Algorithm?**

The Apriori algorithm has three main components:

*   Support
*   Lift
*   Confidence

Here is a tabular representation of this purchase data:

          Milk      Beer      Eggs   Bread  Bananas   Apples
Basket1    1         1          1      1       0       0

Basket2    1         0          0      1       0       0

Basket3    1         0          0      1       0       1

Basket4    0         0          0      1       1       1


Let’s calculate the support, confidence, and lift.

**Support**

The first component of the Apriori algorithm is support – we use it to assess the overall popularity of a given product with the following formula:

Support(item) = Transactions comprising the item / Total            transactions

A high support value indicates that the item is present in most purchases, therefore marketers should focus on it more.

**Confidence**

Confidence tells us the likelihood of different purchase combinations. We calculate that using the following formula:

Confidence (Bread -> Milk) = Transactions comprising bread and milk / Transactions comprising bread

**Lift**

Finally, lift refers to the increase in the ratio of the sale of milk when you sell bread:

Lift = Confidence (Bread -> Milk) / Support(Bread) = 0.75/1 = 1.3.

This means that customers are 1.3 times more likely to buy milk if you also sell bread.

**Step 1: Pre-Requisites for Performing Market Basket Analysis**

Download the dataset "groceries_dataset.csv"

**Step 2: Reading the Dataset**

In [ ]:
import pandas as pd
from google.colab import drive
#drive.mount('/content/drive')
df = pd.read_csv('/content/Groceries_dataset.csv')
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


**Step 3: Data Preparation for Market Basket Analysis**

Before we perform market basket analysis, we need to convert this data into a format that can easily be ingested into the Apriori algorithm. In other words, we need to turn it into a tabular structure comprising ones and zeros, as displayed in the bread and milk example above.

To achieve this, the first group items that have the same member number and date:

In [ ]:
df['single_transaction'] = df['Member_number'].astype(str)+'_'+df['Date'].astype(str)

df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Member_number,Date,itemDescription,single_transaction
0,1808,21-07-2015,tropical fruit,1808_21-07-2015
1,2552,05-01-2015,whole milk,2552_05-01-2015
2,2300,19-09-2015,pip fruit,2300_19-09-2015
3,1187,12-12-2015,other vegetables,1187_12-12-2015
4,3037,01-02-2015,whole milk,3037_01-02-2015


Now, let’s pivot this table to convert the items into columns and the transaction into rows:

In [ ]:
df2 = pd.crosstab(df['single_transaction'], df['itemDescription'])
df2.head()

The final data pre-processing step involves encoding all values in the above data frame to 0 and 1.

In [ ]:
def encode(item_freq):
    res = 0
    if item_freq > 0:
        res = 1
    return res
basket_input = df2.applymap(encode)
print(basket_input)

itemDescription     Instant food products  UHT-milk  abrasive cleaner  \
single_transaction                                                      
1000_15-03-2015                         0         0                 0   
1000_24-06-2014                         0         0                 0   
1000_24-07-2015                         0         0                 0   
1000_25-11-2015                         0         0                 0   
1000_27-05-2015                         0         0                 0   
...                                   ...       ...               ...   
4999_24-01-2015                         0         0                 0   
4999_26-12-2015                         0         0                 0   
5000_09-03-2014                         0         0                 0   
5000_10-02-2015                         0         0                 0   
5000_16-11-2014                         0         0                 0   

itemDescription     artif. sweetener  baby cosmeti

**Step 4: Build the Apriori Algorithm for Market Basket Analysis**

Now, let’s import the Apriori algorithm from the MLXtend Python package and use it to discover frequently-bought-together item combinations:

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(basket_input, min_support=0.001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift")
rules.head()

To get the most frequent item combinations in the entire dataset, let’s sort the dataset by support, confidence, and lift:

In [ ]:
rules.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(8)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
622,(rolls/buns),(whole milk),0.110005,0.157923,0.013968,0.126974,0.804028,-0.003404,0.964550,-0.214986
623,(whole milk),(rolls/buns),0.157923,0.110005,0.013968,0.088447,0.804028,-0.003404,0.976350,-0.224474
694,(yogurt),(whole milk),0.085879,0.157923,0.011161,0.129961,0.822940,-0.002401,0.967861,-0.190525
695,(whole milk),(yogurt),0.157923,0.085879,0.011161,0.070673,0.822940,-0.002401,0.983638,-0.203508
551,(soda),(other vegetables),0.097106,0.122101,0.009691,0.099794,0.817302,-0.002166,0.975219,-0.198448
550,(other vegetables),(soda),0.122101,0.097106,0.009691,0.079365,0.817302,-0.002166,0.980729,-0.202951
649,(sausage),(whole milk),0.060349,0.157923,0.008955,0.148394,0.939663,-0.000575,0.988811,-0.063965
648,(whole milk),(sausage),0.157923,0.060349,0.008955,0.056708,0.939663,-0.000575,0.996140,-0.070851


The resulting table shows that the four most popular product combinations that are frequently bought together are:

Rolls and milk
Yogurt and milk
Sausages and milk
Soda and vegetables
One reason for this could be that the grocery store ran a promotion on these items together or displayed them within the same line of sight to improve sales.